In [31]:
import os
import numpy as np
import mido
NUM_TRACKS = 4
NUM_TIMESHIFTS = 100
TIMESHIFT_LENGTH = 1./NUM_TIMESHIFTS
NUM_MIDI_PITCHES = 128

# Get note sequences for each track.

In [ ]:
def get_tempo(midi):
    """Get tempo from MidiFile object.

    Args:
       midi: MidiFile object.
       
    Returns:
        Tempo in microseconds.
    """
    tempo = -1
    for i, track in enumerate(midi.tracks):
        for msg in track:
            if msg.is_meta and msg.type == 'set_tempo':
                msg_str = str(msg)
                start = msg_str.index('tempo=')
                new_msg = msg_str[start + 6:]
                end = new_msg.index(' ')
                tempo = int(new_msg[:end])

    return tempo

def get_note_sequences(midi_file):
    """Get note sequences for each track from a MidiFile object.

    Args:
       midi_file: Filename.
       
    Returns:
        List of tracks containing note sequences, where each note is a dictionary containing the note type
        ('note_on' or 'note_off'), the MIDI pitch value, and duration of the note in seconds.
    """
    midi = mido.MidiFile(midi_file)
    tempo = get_tempo(midi)
    if tempo == -1:
        return None
    
    tracks = []

    for track in midi.tracks:
        note_sequences = []
        for msg in track:               
            if not msg.is_meta:
                note_vector = {}
                # Convert time to seconds.
                time = mido.tick2second(msg.time, midi.ticks_per_beat, tempo)
                if msg.type == 'note_on' or msg.type == 'note_off':
                    # Get note value.
                    msg_str = str(msg)
                    start = msg_str.index('note=')
                    new_msg = msg_str[start + 5:]
                    end = new_msg.index(' ')
                    note = int(new_msg[:end])

                    # Set note vector values.
                    note_vector["type"] = msg.type
                    note_vector["note"] = note
                    note_vector["time"] = time

                    # Add note vector to note sequence list.
                    note_sequences.append(note_vector)
        tracks.append(note_sequences)

    return tracks

# Get final sequences of vectors.

In [ ]:
def tracks_to_vector_sequence(tracks):
    """Get final vector from note sequence dictionaries.

    Args:
       tracks: List of tracks containing note sequence dictionaries with type (note_on or note_off),
       note, and time delay before note. Tracks are sorted from highest to lowest.
       
    Returns:
        Final one-hot vector containing NUM_MIDI_PITCHES * NUM_TRACKS note_on events,
        NUM_MIDI_PITCHES * NUM_TRACKS note_off events,
        and NUM_TIMESHIFTS timeshift events in intervals of 10 ms each.
    """
    if tracks == None:
        return []
    
    assert len(tracks) <= NUM_TRACKS + 1
    
    # Sort all events by start time.
    events = []
    for i, sequence in enumerate(tracks):
        start_time = 0
        for event in sequence:
            start_time += event["time"]
            new_event = {"type": event["type"], "track": i, "note": event["note"],
                         "start_time": start_time}
            events.append(new_event)
    events = sorted(events, key=lambda x: x["start_time"])
    
    # Fix time lengths.
    prev_start_time = 0
    for e in events:
        e["time"] = e["start_time"] - prev_start_time
        prev_start_time = e["start_time"]
    
    # Create final vector sequence.
    final_sequence = []
    for e in events:
        time_vector = [0] * (NUM_MIDI_PITCHES * 2 * NUM_TRACKS + NUM_TIMESHIFTS)
        
        # Create rest vectors if time is greater than 1 second.
        if e["time"] > 0:
            time = e["time"]
            while time > 1.0:
                rest_vector = [0] * (NUM_MIDI_PITCHES * 2 * NUM_TRACKS + NUM_TIMESHIFTS)
                rest_vector[-1] = 1
                final_sequence.append(rest_vector)
                time -= 1.0
            timeshift = int(np.ceil(time / TIMESHIFT_LENGTH) - 1)
            time_vector[NUM_MIDI_PITCHES * 2 * NUM_TRACKS + timeshift] = 1
            final_sequence.append(time_vector)
        
        note_vector = [0] * (NUM_MIDI_PITCHES * 2 * NUM_TRACKS + NUM_TIMESHIFTS)
        track_offset = NUM_MIDI_PITCHES * e["track"]
        # Set correct note.
        if e["type"] == 'note_on':
            note_vector[track_offset + e["note"]] = 1
        else:
            note_vector[NUM_MIDI_PITCHES * NUM_TRACKS + track_offset + e["note"]] = 1
        final_sequence.append(note_vector)
        
    return final_sequence

# Use the functions below to get midi to vector sequence.

In [28]:
def midi_to_vector(filename):
    """Get final vector from midi file.

    Args:
       filename: File name of midi.
       
    Returns:
        Final one-hot vector containing NUM_MIDI_PITCHES * NUM_TRACKS note_on events,
        NUM_MIDI_PITCHES * NUM_TRACKS note_off events,
        and NUM_TIMESHIFTS timeshift events in intervals of 10 ms each.
    """
    tracks = get_note_sequences(filename)
    return tracks_to_vector_sequence(tracks)

In [29]:
def get_training_data():
    """Generate training data array for all files in "midis_processed/" directory.
       
    Returns:
        Numpy array of training data.
    """
    training_data = []
    for filename in os.listdir('midis_processed/'):
        if filename.endswith(".mid"):
            print(filename)
            training_data.append(midi_to_vector('midis_processed/' + filename))
    return np.array(training_data)

In [30]:
training_data = get_training_data()

1999.mid
elm_street_metal.mid
Bbtitle2.mid
bohmagna.mid
MKMINE.mid
T_Tsub29.mid
1943lost (1).mid
Astyanax-Redwood.mid
bomb2stg1.mid
T_Rygar7.mid
Amagon-Final_Boss.mid
T_Ryga11.mid
Lolo2clr.mid
Battletoads_Pause_Music.mid
alphamis.mid
MKSPACE.mid
Adventure_Island_II-Overworld_3.mid
Lolo3lvl.mid
Btsurf.mid
Anticipation12.mid
archon_battle.mid
Volcano_World.mid
Burai_Fighter_-_Taxan.mid
T_Ryga10.mid
Illuminator.mid
bohgreen.mid
ANTARIntro.mid
Bub_Invincible.mid
T_Tsub14.mid
armadillo_casino.mid
T_Batman_Stage2-1.mid
T_Tsub28.mid
BTDDLvl2.mid
Peloponnesus.mid
MKTITLE.mid
bbtitler.mid
Action52CityofDoom.mid
bc5.mid
airwolf_title.mid
AI3-Thundercloud.mid
Action52DamBusters.mid
Athena-Boss.mid
T_Tsub16.mid
Adventure_Island_II-Credits.mid
T_Rygar4.mid
Amagon-Zone_4_Rain_Forest.mid
basement.mid
palace2.mid
AFLevel_Intro.mid
bb-Ending.mid
Adventure_Island_II-Bonus.mid
Level_8_-_Intruder_Excluder.mid
T_Ryga12.mid
Athena-Hell.mid
Anticipation10.mid
Anticipation11.mid
Adventure_Island_II-Overworld_

A3_cave.mid
T_Tsub24.mid
Bionic_Commando_Level_Clear_Theme.mid
BoS_Intermission_Mini-Game.mid
T_Tsub30.mid
Athena-Ice.mid
B2_Stage_6.mid
